<a href="https://colab.research.google.com/github/Nukaraju2003/Natural-Language-Processing/blob/main/Audio_to_text_Pipeline_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-jdlg0ll4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-jdlg0ll4
  Resolved https://github.com/openai/whisper.git to commit fc5ded7d9045c693692f13853857c3f8baea3a7b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803579 sha256=757b7b94e3e2528aa3fc25fb43ea7b8d440b1a290e88b1016ad0001833750581
  Stored in directory: /tmp/pip-ephem-wheel-cache-n_jropb3/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [ ]:
!pip install transformers

In [ ]:
!pip install pyloudnorm

In [ ]:
!pip install pydub

In [ ]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.4 MB/s eta 0:00:00


In [ ]:
# Import required libraries
import whisper
import warnings
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer
warnings.filterwarnings('ignore')

In [ ]:
# Load the Whisper model (choose 'small', 'medium', or 'large' for better accuracy)
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:43<00:00, 71.0MiB/s]


In [ ]:
# Translation models and tokenizers setup
# NLLB for low-resource language pairs (Hindi ↔ Telugu, Telugu ↔ Hindi)
nllb_model_name = "facebook/nllb-200-distilled-1.3B"
nllb_tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model_name)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
# Helsinki-NLP for English ↔ Hindi
helsinki_en_hi_model_name = "Helsinki-NLP/opus-mt-en-hi"
helsinki_en_hi_tokenizer = MarianTokenizer.from_pretrained(helsinki_en_hi_model_name)
helsinki_en_hi_model = MarianMTModel.from_pretrained(helsinki_en_hi_model_name)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

In [ ]:
# Helper function to split long text into chunks
def split_text_into_chunks(text, max_tokens, tokenizer):
    tokens = tokenizer.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk_tokens, skip_special_tokens=True))
    return chunks

In [ ]:
# Helper function to translate chunks of text
def translate_chunks(chunks, translation_function):
    translated_chunks = [translation_function(chunk) for chunk in chunks]
    return " ".join(translated_chunks)

In [ ]:
# Wrapper function for large text translation
def translate_large_text(text, translation_function, tokenizer, max_tokens=512):
    chunks = split_text_into_chunks(text, max_tokens, tokenizer)
    translated_text = translate_chunks(chunks, translation_function)
    return translated_text

In [ ]:
def hindi_to_english(text):
    src_lang = "hin_Deva"  # Hindi
    tgt_lang = "eng_Latn"  # English
    nllb_tokenizer.src_lang = src_lang
    inputs = nllb_tokenizer(text, return_tensors="pt", padding=True)
    outputs = nllb_model.generate(**inputs, forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids(tgt_lang))
    return nllb_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def english_to_hindi(text):
    inputs = helsinki_en_hi_tokenizer(text, return_tensors="pt", padding=True)
    outputs = helsinki_en_hi_model.generate(**inputs)
    return helsinki_en_hi_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def english_to_telugu(text):
    src_lang = "eng_Latn"
    tgt_lang = "tel_Telu"
    nllb_tokenizer.src_lang = src_lang
    inputs = nllb_tokenizer(text, return_tensors="pt", padding=True)
    outputs = nllb_model.generate(**inputs, forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids(tgt_lang))
    return nllb_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def telugu_to_english(text):
    src_lang = "tel_Telu"
    tgt_lang = "eng_Latn"
    nllb_tokenizer.src_lang = src_lang
    inputs = nllb_tokenizer(text, return_tensors="pt", padding=True)
    outputs = nllb_model.generate(**inputs, forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids(tgt_lang))
    return nllb_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def telugu_to_hindi(text):
    src_lang = "tel_Telu"
    tgt_lang = "hin_Deva"
    nllb_tokenizer.src_lang = src_lang
    inputs = nllb_tokenizer(text, return_tensors="pt", padding=True)
    outputs = nllb_model.generate(**inputs, forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids(tgt_lang))
    return nllb_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def hindi_to_telugu(text):
    src_lang = "hin_Deva"
    tgt_lang = "tel_Telu"
    nllb_tokenizer.src_lang = src_lang
    inputs = nllb_tokenizer(text, return_tensors="pt", padding=True)
    outputs = nllb_model.generate(**inputs, forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids(tgt_lang))
    return nllb_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def transcribe_and_translate(audio_path):
    # Step 1: Transcribe the audio file
    print("Transcribing audio...")
    result = model.transcribe(audio_path)
    transcribed_text = result["text"]
    print("Transcription Complete!")
    print("Transcribed Text:", transcribed_text)

    # Step 2: User Translation Choice
    print("\nSelect a translation option:")
    print("1. Hindi to English")
    print("2. English to Hindi")
    print("3. English to Telugu")
    print("4. Telugu to English")
    print("5. Telugu to Hindi")
    print("6. Hindi to Telugu")

    choice = int(input("Enter your choice (1-6): "))

    # Step 3: Perform Translation
    if choice == 1:
        translated_text = translate_large_text(transcribed_text, hindi_to_english, nllb_tokenizer)
    elif choice == 2:
        translated_text = translate_large_text(transcribed_text, english_to_hindi, helsinki_en_hi_tokenizer)
    elif choice == 3:
        translated_text = translate_large_text(transcribed_text, english_to_telugu, nllb_tokenizer)
    elif choice == 4:
        translated_text = translate_large_text(transcribed_text, telugu_to_english, nllb_tokenizer)
    elif choice == 5:
        translated_text = translate_large_text(transcribed_text, telugu_to_hindi, nllb_tokenizer)
    elif choice == 6:
        translated_text = translate_large_text(transcribed_text, hindi_to_telugu, nllb_tokenizer)
    else:
        print("Invalid choice. Please try again.")
        return

    # Step 4: Display Translated Text
    print("Translated Text:")
    print(translated_text)

In [ ]:
# Specify the audio file path
audio_path = "/content/audio (1).mp3"  # Replace with your file path

In [ ]:
print(audio_path)

In [ ]:
# Run the pipeline
transcribe_and_translate(audio_path)